In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time
from datetime import datetime as dt
from torchvision import datasets, transforms
import pickle
!pip install torchattacks==3.3.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Konstante za ponovne treninge (da bude uvek isto)

In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(42)

# Konfiguracija modela

In [ ]:
OMEGA = 1
HIDDEN = 500
CONV_OUT = 4 * 4 * 50
NUM_CLASSES = 47

LR = 0.01
MOM = 0.5

# Glavna klasa modela

In [ ]:
from torchvision.transforms.transforms import ToTensor
class Net(nn.Module):
    def __init__(self, net_type):
        super(Net, self).__init__()

        self.net_type = net_type

        if ('normal' in self.net_type) or ('hybrid_nor' in self.net_type) or ('synergy_nor' in self.net_type) or ('synergy_all' in self.net_type):
           self.conv1_normal = nn.Conv2d(1, 20, 5, 1)
           self.pool_normal = nn.MaxPool2d(2, 2)
           self.conv2_normal = nn.Conv2d(20, 50, 5, 1)

        if ('negative' in self.net_type) or ('hybrid_neg' in self.net_type) or ('synergy_neg' in self.net_type) or ('synergy_all' in self.net_type):
           self.conv1_negative = nn.Conv2d(1, 20, 5, 1)
           self.pool_negative = nn.MaxPool2d(2, 2)
           self.conv2_negative = nn.Conv2d(20, 50, 5, 1)

        if ('normal' in self.net_type) or ('synergy_nor' in self.net_type) or ('synergy_all' in self.net_type):
           self.fc1_normal = nn.Linear(CONV_OUT, HIDDEN)
           self.fc2_normal = nn.Linear(HIDDEN, NUM_CLASSES)

        if ('hybrid_nor' in self.net_type) or ('synergy_nor' in self.net_type) or ('synergy_all' in self.net_type):
           self.fc1_normal_n = nn.Linear(CONV_OUT, HIDDEN)
           self.fc2_normal_n = nn.Linear(HIDDEN, NUM_CLASSES)

        if ('hybrid_neg' in self.net_type) or ('synergy_neg' in self.net_type) or ('synergy_all' in self.net_type):
           self.fc1_negative = nn.Linear(CONV_OUT, HIDDEN)
           self.fc2_negative = nn.Linear(HIDDEN, NUM_CLASSES)

        if ('negative' in self.net_type) or ('synergy_neg' in self.net_type) or ('synergy_all' in self.net_type):
           self.fc1_negative_n = nn.Linear(CONV_OUT, HIDDEN)
           self.fc2_negative_n = nn.Linear(HIDDEN, NUM_CLASSES)

    def forward(self, x):
        # conv block:
        if ('normal' in self.net_type) or ('hybrid_nor' in self.net_type) or ('synergy_nor' in self.net_type) or ('synergy_all' in self.net_type):
           x_normal = self.pool_normal(F.relu(self.conv1_normal(x)))
           x_normal = self.pool_normal(F.relu(self.conv2_normal(x_normal)))
           x_normal = x_normal.view(-1, CONV_OUT)

        if ('negative' in self.net_type) or ('hybrid_neg' in self.net_type) or ('synergy_neg' in self.net_type) or ('synergy_all' in self.net_type):
           x_negative = self.pool_negative(F.relu(self.conv1_negative(x)))
           x_negative = self.pool_negative(F.relu(self.conv2_negative(x_negative)))
           x_negative = x_negative.view(-1, CONV_OUT)

        # fc block:
        if 'synergy_nor' in self.net_type:
            x_normal_normal = self.fc2_normal(F.relu(self.fc1_normal(x_normal)))
            x_normal_negative = self.fc2_normal_n(F.relu(self.fc1_normal_n(1 - x_normal)))

            o = x_normal_normal + x_normal_negative * OMEGA

        else:
            if 'synergy_neg' in self.net_type:
                x_negative_normal = self.fc2_negative(F.relu(self.fc1_negative(x_negative)))
                x_negative_negative = self.fc2_negative_n(F.relu(self.fc1_negative_n(1 - x_negative)))

                o = x_negative_normal + x_negative_negative * OMEGA

            else:
                if 'synergy_all' in self.net_type:
                    x_normal_normal = self.fc2_normal(F.relu(self.fc1_normal(x_normal)))
                    x_normal_negative = self.fc2_normal_n(F.relu(self.fc1_normal_n(1 - x_normal)))
                    x_negative_normal = self.fc2_negative(F.relu(self.fc1_negative(x_negative)))
                    x_negative_negative = self.fc2_negative_n(F.relu(self.fc1_negative_n(1 - x_negative)))

                    o = x_normal_normal + x_normal_negative * OMEGA + x_negative_normal + x_negative_negative * OMEGA
                else:
                    if 'negative' in self.net_type:
                         x = 1 - x_negative
                         o = self.fc2_negative_n(F.relu(self.fc1_negative_n(x)))
                    else:
                         if 'normal' in self.net_type:
                             x = x_normal
                             o = self.fc2_normal(F.relu(self.fc1_normal(x)))
                         else:
                             if 'hybrid_neg' in self.net_type:
                                 x = x_negative
                                 o = self.fc2_negative(F.relu(self.fc1_negative(x)))
                             else:
                                 x = 1 - x_normal
                                 o = self.fc2_normal_n(F.relu(self.fc1_normal_n(x)))

        return F.log_softmax(o, dim=1)

# Pomoćne funkcije za trening i testove

In [ ]:
def train(model, device, train_loader, optimizer, epoch, loss_fn=F.nll_loss):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
                  .format(model.net_type, epoch, batch_idx * len(data),
                          len(train_loader.dataset),
                          100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, loss_fn=F.nll_loss, dataset_name=None, filter_map=None):
    model.eval()
    test_loss = 0
    correct = 0
    index_store = []
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            if filter_map and dataset_name in filter_map:
              for batch in data:
                for channel in batch:
                  channel.mul_(filter_map[dataset_name].to(device))
            output = model(data)
            test_loss += loss_fn(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            c = pred.eq(target.view_as(pred)).sum().item()
            correct += c
            index_store.append((i, c))

    test_loss /= len(test_loader.dataset)

    print('[{}] Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'
          .format(model.net_type, test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))

    return index_store

# Učitavanje podataka (i "stari" filteri)

In [ ]:
kwargs = {'num_workers': 2, 'pin_memory': True} \
         if torch.cuda.is_available() else {}

f_v, f_h, f_d, f_t = torch.ones((28, 28)), torch.ones((28, 28)), \
                     torch.ones((28, 28)), torch.ones((28, 28))
f_v[:, :14] = 0  # vertical filter
f_h[:14, :] = 0  # horizontal filter
f_d[:14, 15:], f_d[15:, :14] = 0, 0  # diagonal filter
f_t[6:15, 6:15], f_t[18:27, 11:20], f_t[8:17, 17:26] = 0, 0, 0  # tcut


def mnist_loader(train=False):
    return torch.utils.data.DataLoader(
        datasets.MNIST('../data', download=True, train=train,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64 if train else 1, shuffle=train, **kwargs)


def emnist_loader(split='balanced', batch_size=64):
    def _loader(train=False, batch_size=64):
        return torch.utils.data.DataLoader(
           datasets.EMNIST('../data', split=split, download=True, train=train,
                           transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                            ])), batch_size=batch_size, shuffle=train, **kwargs)
    return _loader

transform_cifar = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def cifar10_loader(train=False, batch_size=4):
    return torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', download=True, train=train,
                          transform=transform_cifar),
                          batch_size=batch_size,
                          shuffle=train, **kwargs)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

def cifar10_loader_resnet(train=False, batch_size=1536):
    return torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', download=True, train=train,
                          transform=transform_train if train else transform_test),
                          batch_size=batch_size,
                          shuffle=train, **kwargs)

cifar10_classes = ['plane', 'car', 'bird', 'cat',
                   'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Gde se radi trening (GPU)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Trening modela

In [ ]:
loader = emnist_loader()

train_loader = loader(train=True)
test_loader = loader()

model_normal = Net('normal').to(device)
model_negative = Net('negative').to(device)

model_synergy_nor = Net('synergy_nor').to(device)
model_synergy_neg = Net('synergy_neg').to(device)
model_synergy_all = Net('synergy_all').to(device)

model_hybrid_nor = Net('hybrid_nor').to(device)
model_hybrid_neg = Net('hybrid_neg').to(device)

model_tr_synergy_all = Net('tr_synergy_all').to(device)

# Obični model

In [ ]:
# ---- Normal net:

optimizer_normal = optim.SGD(filter(lambda p: p.requires_grad,
                             model_normal.parameters()), lr=LR, momentum=MOM)

for epoch in range(1, 10 + 1):
    train(model_normal, device, train_loader, optimizer_normal,
          epoch, loss_fn=F.cross_entropy)

for param in model_normal.parameters():
  param.requires_grad = False

conv1_nor = model_normal.conv1_normal
conv2_nor = model_normal.conv2_normal

# Zamrzavanje slojeva
conv1_nor.weight.requires_grad = False
conv2_nor.weight.requires_grad = False
conv1_nor.bias.requires_grad = False
conv2_nor.bias.requires_grad = False



[normal] Train Epoch: 1 [0/112800 (0%)]	Loss: 3.850885
[normal] Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.695619
[normal] Train Epoch: 2 [0/112800 (0%)]	Loss: 0.462280
[normal] Train Epoch: 2 [64000/112800 (57%)]	Loss: 0.496040
[normal] Train Epoch: 3 [0/112800 (0%)]	Loss: 0.297183
[normal] Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.379306
[normal] Train Epoch: 4 [0/112800 (0%)]	Loss: 0.409661
[normal] Train Epoch: 4 [64000/112800 (57%)]	Loss: 0.316735
[normal] Train Epoch: 5 [0/112800 (0%)]	Loss: 0.468509
[normal] Train Epoch: 5 [64000/112800 (57%)]	Loss: 0.497590
[normal] Train Epoch: 6 [0/112800 (0%)]	Loss: 0.581184
[normal] Train Epoch: 6 [64000/112800 (57%)]	Loss: 0.308109
[normal] Train Epoch: 7 [0/112800 (0%)]	Loss: 0.298122
[normal] Train Epoch: 7 [64000/112800 (57%)]	Loss: 0.415047
[normal] Train Epoch: 8 [0/112800 (0%)]	Loss: 0.224063
[normal] Train Epoch: 8 [64000/112800 (57%)]	Loss: 0.224526
[normal] Train Epoch: 9 [0/112800 (0%)]	Loss: 0.225632
[normal] Train Epoch: 9 [

#Snimanje modela

In [ ]:
torch.save(model_normal.state_dict(), 'model_normal.pt')

#Učitavanje modela

In [ ]:
model_normal = Net('normal').to(device)
model_normal.load_state_dict(torch.load('model_normal.pt'))
# model_normal.load_state_dict(torch.load('model_normal.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

#Štampanje težina

In [ ]:
with torch.no_grad():
   print(conv1_nor.weight)


# Negativni (hibrid) model

In [ ]:
# ---- Hybrid net:

# Deljenje zamrznutih slojeva
model_hybrid_nor.conv1_normal = conv1_nor
model_hybrid_nor.conv2_normal = conv2_nor

optimizer_hybrid_nor = optim.SGD(filter(lambda p: p.requires_grad,
                             model_hybrid_nor.parameters()), lr=LR, momentum=MOM)

for epoch in range(1, 10 + 1):
    train(model_hybrid_nor, device, train_loader, optimizer_hybrid_nor,
          epoch, loss_fn=F.cross_entropy)

for param in model_hybrid_nor.parameters():
  param.requires_grad = False


[hybrid_nor] Train Epoch: 1 [0/112800 (0%)]	Loss: 4.365130
[hybrid_nor] Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.594000
[hybrid_nor] Train Epoch: 2 [0/112800 (0%)]	Loss: 0.556093
[hybrid_nor] Train Epoch: 2 [64000/112800 (57%)]	Loss: 0.262030
[hybrid_nor] Train Epoch: 3 [0/112800 (0%)]	Loss: 0.258823
[hybrid_nor] Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.163162
[hybrid_nor] Train Epoch: 4 [0/112800 (0%)]	Loss: 0.420351
[hybrid_nor] Train Epoch: 4 [64000/112800 (57%)]	Loss: 0.305534
[hybrid_nor] Train Epoch: 5 [0/112800 (0%)]	Loss: 0.275575
[hybrid_nor] Train Epoch: 5 [64000/112800 (57%)]	Loss: 0.292623
[hybrid_nor] Train Epoch: 6 [0/112800 (0%)]	Loss: 0.282821
[hybrid_nor] Train Epoch: 6 [64000/112800 (57%)]	Loss: 0.376338
[hybrid_nor] Train Epoch: 7 [0/112800 (0%)]	Loss: 0.185545
[hybrid_nor] Train Epoch: 7 [64000/112800 (57%)]	Loss: 0.246094
[hybrid_nor] Train Epoch: 8 [0/112800 (0%)]	Loss: 0.193338
[hybrid_nor] Train Epoch: 8 [64000/112800 (57%)]	Loss: 0.137952
[hybrid_nor] Tra

#Snimanje modela

In [ ]:
torch.save(model_hybrid_nor.state_dict(), 'model_hybrid_nor.pt')

#Učitavanje modela

In [ ]:
model_hybrid_nor = Net('hybrid_nor').to(device)
model_hybrid_nor.load_state_dict(torch.load('model_hybrid_nor.pt'))
# model_hybrid_nor.load_state_dict(torch.load('model_hybrid_nor.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

# Skroz negativni model

In [ ]:
# ---- Negative net:

optimizer_negative = optim.SGD(filter(lambda p: p.requires_grad,
                             model_negative.parameters()), lr=LR, momentum=MOM)

for epoch in range(1, 10 + 1):
    train(model_negative, device, train_loader, optimizer_negative,
          epoch, loss_fn=F.cross_entropy)

for param in model_negative.parameters():
  param.requires_grad = False

conv1_neg = model_negative.conv1_negative
conv2_neg = model_negative.conv2_negative

conv1_neg.weight.requires_grad = False
conv2_neg.weight.requires_grad = False
conv1_neg.bias.requires_grad = False
conv2_neg.bias.requires_grad = False

[negative] Train Epoch: 1 [0/112800 (0%)]	Loss: 3.838741
[negative] Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.695061
[negative] Train Epoch: 2 [0/112800 (0%)]	Loss: 0.563996
[negative] Train Epoch: 2 [64000/112800 (57%)]	Loss: 0.413599
[negative] Train Epoch: 3 [0/112800 (0%)]	Loss: 0.459479
[negative] Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.541164
[negative] Train Epoch: 4 [0/112800 (0%)]	Loss: 0.354509
[negative] Train Epoch: 4 [64000/112800 (57%)]	Loss: 0.280990
[negative] Train Epoch: 5 [0/112800 (0%)]	Loss: 0.188881
[negative] Train Epoch: 5 [64000/112800 (57%)]	Loss: 0.450622
[negative] Train Epoch: 6 [0/112800 (0%)]	Loss: 0.228978
[negative] Train Epoch: 6 [64000/112800 (57%)]	Loss: 0.287582
[negative] Train Epoch: 7 [0/112800 (0%)]	Loss: 0.215137
[negative] Train Epoch: 7 [64000/112800 (57%)]	Loss: 0.159629
[negative] Train Epoch: 8 [0/112800 (0%)]	Loss: 0.468343
[negative] Train Epoch: 8 [64000/112800 (57%)]	Loss: 0.119397
[negative] Train Epoch: 9 [0/112800 (0%)]	Loss: 

#Snimanje modela

In [ ]:
torch.save(model_negative.state_dict(), 'model_negative.pt')

#Učitavanje modela

In [ ]:
model_negative = Net('negative').to(device)
model_negative.load_state_dict(torch.load('model_negative.pt'))
# model_negative.load_state_dict(torch.load('model_negative.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

# Skroz negativni (hibrid) model

In [ ]:
# ---- Hybrid_neg net:

model_hybrid_neg.conv1_negative = conv1_neg
model_hybrid_neg.conv2_negative = conv2_neg

optimizer_hybrid_neg = optim.SGD(filter(lambda p: p.requires_grad,
                             model_hybrid_neg.parameters()), lr=LR, momentum=MOM)

for epoch in range(1, 10 + 1):
    train(model_hybrid_neg, device, train_loader, optimizer_hybrid_neg,
          epoch, loss_fn=F.cross_entropy)

for param in model_hybrid_neg.parameters():
  param.requires_grad = False


[hybrid_neg] Train Epoch: 1 [0/112800 (0%)]	Loss: 4.013987
[hybrid_neg] Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.449314
[hybrid_neg] Train Epoch: 2 [0/112800 (0%)]	Loss: 0.459528
[hybrid_neg] Train Epoch: 2 [64000/112800 (57%)]	Loss: 0.451301
[hybrid_neg] Train Epoch: 3 [0/112800 (0%)]	Loss: 0.405234
[hybrid_neg] Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.273954
[hybrid_neg] Train Epoch: 4 [0/112800 (0%)]	Loss: 0.281573
[hybrid_neg] Train Epoch: 4 [64000/112800 (57%)]	Loss: 0.344358
[hybrid_neg] Train Epoch: 5 [0/112800 (0%)]	Loss: 0.155168
[hybrid_neg] Train Epoch: 5 [64000/112800 (57%)]	Loss: 0.485289
[hybrid_neg] Train Epoch: 6 [0/112800 (0%)]	Loss: 0.412850
[hybrid_neg] Train Epoch: 6 [64000/112800 (57%)]	Loss: 0.209984
[hybrid_neg] Train Epoch: 7 [0/112800 (0%)]	Loss: 0.263619
[hybrid_neg] Train Epoch: 7 [64000/112800 (57%)]	Loss: 0.302772
[hybrid_neg] Train Epoch: 8 [0/112800 (0%)]	Loss: 0.222865
[hybrid_neg] Train Epoch: 8 [64000/112800 (57%)]	Loss: 0.343343
[hybrid_neg] Tra

#Snimanje modela

In [ ]:
torch.save(model_hybrid_neg.state_dict(), 'model_hybrid_neg.pt')

#Učitavanje modela

In [ ]:
model_hybrid_neg = Net('hybrid_neg').to(device)
model_hybrid_neg.load_state_dict(torch.load('model_hybrid_neg.pt'))
# model_hybrid_neg.load_state_dict(torch.load('model_hybrid_neg.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

# Sinergije, bez treninga

In [ ]:
# ---- synergy_nor net (not trained):

model_synergy_nor.conv1_normal = conv1_nor
model_synergy_nor.conv2_normal = conv2_nor

model_synergy_nor.fc1_normal = model_normal.fc1_normal
model_synergy_nor.fc2_normal = model_normal.fc2_normal

model_synergy_nor.fc1_normal_n = model_hybrid_nor.fc1_normal_n
model_synergy_nor.fc2_normal_n = model_hybrid_nor.fc2_normal_n

# ---- synergy_neg net (not trained):

model_synergy_neg.conv1_negative = conv1_neg
model_synergy_neg.conv2_negative = conv2_neg

model_synergy_neg.fc1_negative = model_hybrid_neg.fc1_negative
model_synergy_neg.fc2_negative = model_hybrid_neg.fc2_negative

model_synergy_neg.fc1_negative_n = model_negative.fc1_negative_n
model_synergy_neg.fc2_negative_n = model_negative.fc2_negative_n

# ---- synergy_all net (not trained):

model_synergy_all.conv1_normal = conv1_nor
model_synergy_all.conv2_normal = conv2_nor
model_synergy_all.conv1_negative = conv1_neg
model_synergy_all.conv2_negative = conv2_neg

model_synergy_all.fc1_normal = model_normal.fc1_normal
model_synergy_all.fc2_normal = model_normal.fc2_normal

model_synergy_all.fc1_normal_n = model_hybrid_nor.fc1_normal_n
model_synergy_all.fc2_normal_n = model_hybrid_nor.fc2_normal_n

model_synergy_all.fc1_negative = model_hybrid_neg.fc1_negative
model_synergy_all.fc2_negative = model_hybrid_neg.fc2_negative

model_synergy_all.fc1_negative_n = model_negative.fc1_negative_n
model_synergy_all.fc2_negative_n = model_negative.fc2_negative_n


#Snimanje modela

In [ ]:
torch.save(model_synergy_nor.state_dict(), 'model_synergy_nor.pt')
torch.save(model_synergy_neg.state_dict(), 'model_synergy_neg.pt')
torch.save(model_synergy_all.state_dict(), 'model_synergy_all.pt')

#Učitavanje modela

In [ ]:
model_synergy_nor = Net('synergy_nor').to(device)
model_synergy_nor.load_state_dict(torch.load('model_synergy_nor.pt'))
# model_synergy_nor.load_state_dict(torch.load('model_synergy_nor.pt', map_location=torch.device('cpu')))
model_synergy_neg = Net('synergy_neg').to(device)
model_synergy_neg.load_state_dict(torch.load('model_synergy_neg.pt'))
# model_synergy_neg.load_state_dict(torch.load('model_synergy_neg.pt', map_location=torch.device('cpu')))
model_synergy_all = Net('synergy_all').to(device)
model_synergy_all.load_state_dict(torch.load('model_synergy_all.pt'))
# model_synergy_all.load_state_dict(torch.load('model_synergy_all.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

#Trenirana sinergija

In [ ]:
# ---- Trained synergy_all TF:

model_tr_synergy_all.conv1_normal = conv1_nor
model_tr_synergy_all.conv2_normal = conv2_nor
model_tr_synergy_all.conv1_negative = conv1_neg
model_tr_synergy_all.conv2_negative = conv2_neg

optimizer_tr_synergy_all = optim.SGD(filter(lambda p: p.requires_grad,
                                    model_tr_synergy_all.parameters()),
                                    lr=LR, momentum=MOM)

for epoch in range(1, 10 + 1):
    train(model_tr_synergy_all, device, train_loader,
          optimizer_tr_synergy_all, epoch, loss_fn=F.cross_entropy)


[tr_synergy_all] Train Epoch: 1 [0/112800 (0%)]	Loss: 6.024405
[tr_synergy_all] Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.256137
[tr_synergy_all] Train Epoch: 2 [0/112800 (0%)]	Loss: 0.387234
[tr_synergy_all] Train Epoch: 2 [64000/112800 (57%)]	Loss: 0.249035
[tr_synergy_all] Train Epoch: 3 [0/112800 (0%)]	Loss: 0.394568
[tr_synergy_all] Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.308688
[tr_synergy_all] Train Epoch: 4 [0/112800 (0%)]	Loss: 0.325880
[tr_synergy_all] Train Epoch: 4 [64000/112800 (57%)]	Loss: 0.206799
[tr_synergy_all] Train Epoch: 5 [0/112800 (0%)]	Loss: 0.190261
[tr_synergy_all] Train Epoch: 5 [64000/112800 (57%)]	Loss: 0.299697
[tr_synergy_all] Train Epoch: 6 [0/112800 (0%)]	Loss: 0.160542
[tr_synergy_all] Train Epoch: 6 [64000/112800 (57%)]	Loss: 0.300768
[tr_synergy_all] Train Epoch: 7 [0/112800 (0%)]	Loss: 0.120248
[tr_synergy_all] Train Epoch: 7 [64000/112800 (57%)]	Loss: 0.205827
[tr_synergy_all] Train Epoch: 8 [0/112800 (0%)]	Loss: 0.254975
[tr_synergy_all] Tra

#Snimanje modela

In [ ]:
torch.save(model_tr_synergy_all.state_dict(), 'model_tr_synergy_all.pt')

#Učitavanje modela

In [ ]:
model_tr_synergy_all = Net('tr_synergy_all').to(device)
model_tr_synergy_all.load_state_dict(torch.load('model_tr_synergy_all.pt'))
# model_tr_synergy_all.load_state_dict(torch.load('model_tr_synergy_all.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

# Nova sečenja

In [ ]:
import random
random.seed(0)

def corner_kernel(factor):
    kernel = np.ones((28, 28))
    limit = int(factor * kernel.shape[0]) * 2
    f = (1 - np.tril(kernel[:limit, :limit]))
    kernel[kernel.shape[0] - limit:limit + kernel.shape[0], :limit] = f
    return torch.tensor(kernel)

def single_square_kernel(start=5, size=9):
    kernel = np.ones((28, 28))
    kernel[start:start+size, start:start+size] = 0.
    return torch.tensor(kernel)

def single_square_kernel_random(start=5, size=9):
    random_pos = random.randint(0, 27 - size)
    kernel = np.ones((28, 28))
    kernel[random_pos:random_pos + size, random_pos:random_pos + size] = 0.
    return torch.tensor(kernel)

def multiple_square_kernel(n_cuts=3, start=5, size=9):
    size_triple = size // n_cuts
    kernel = np.ones((28, 28))

    beg = start
    end = start + size_triple

    for _ in range(n_cuts):

        kernel[beg:end, beg:end] = 0.

        beg = end + 5
        end = beg + size_triple

    return torch.tensor(kernel)

def multiple_square_kernel_random(n_cuts = 2, start=0, size=18, sep = 5):
    size_triple = size // n_cuts
    kernel = np.ones((28, 28))

    beg1 = start + random.randint(1, sep)
    end1 = beg1 + size_triple

    beg2 = start + random.randint(1, sep)
    end2 = beg2 + size_triple

    for _ in range(n_cuts):

        kernel[beg1:end1, beg2:end2] = 0.

        beg1 = end1 + random.randint(1, sep)
        end1 = beg1 + size_triple

        beg2 = end2 + random.randint(1, sep)
        end2 = beg2 + size_triple

    return torch.tensor(kernel)

# Testiranje sa svim modelima i svim "novim" sečenjima

In [ ]:
from torchvision import datasets, transforms
import random
random.seed(0)

models = [model_normal, model_negative, model_hybrid_nor, model_hybrid_neg, model_synergy_nor, model_synergy_neg, model_synergy_all, model_tr_synergy_all]


dataset_names = ['Normal', 'C1', 'C2', 'C3', 'SSK', 'SSKR', 'MSK', 'MSKR']

alldatasets = []
for _ in dataset_names:
    alldatasets.append(loader())

filters_map = {'C1': corner_kernel(0.1),
               'C2': corner_kernel(0.2), 'C3': corner_kernel(0.3),
               'SSK': single_square_kernel(), 'SSKR': single_square_kernel_random(), 'MSK': multiple_square_kernel(), 'MSKR': multiple_square_kernel_random()}

for dataset, name in zip(alldatasets, dataset_names):
    print('Testing -- ' + name)
    fdir = '.'
    for m in models:
        index_store = test(m, device, dataset, loss_fn=F.cross_entropy, dataset_name=name, filter_map=filters_map)
        file_name = f'{fdir}/{m.net_type}-{name}-{dt.now()}.pth'
        with open(file_name, 'wb') as f:
            pickle.dump(index_store, f)

for m in models:
    file_name = f'{m.net_type}-{dt.now()}.pth'
    torch.save(m.state_dict(), file_name)

Testing -- Normal
[normal] Test set: Average loss: 0.3730, Accuracy: 16407/18800 (87%)
[negative] Test set: Average loss: 0.3602, Accuracy: 16485/18800 (88%)
[hybrid_nor] Test set: Average loss: 0.3678, Accuracy: 16480/18800 (88%)
[hybrid_neg] Test set: Average loss: 0.3656, Accuracy: 16450/18800 (88%)
[synergy_nor] Test set: Average loss: 0.5036, Accuracy: 16593/18800 (88%)
[synergy_neg] Test set: Average loss: 0.4854, Accuracy: 16590/18800 (88%)
[synergy_all] Test set: Average loss: 0.8516, Accuracy: 16636/18800 (88%)
[tr_synergy_all] Test set: Average loss: 0.3687, Accuracy: 16443/18800 (87%)
Testing -- C1
[normal] Test set: Average loss: 0.3730, Accuracy: 16407/18800 (87%)
[negative] Test set: Average loss: 0.3602, Accuracy: 16485/18800 (88%)
[hybrid_nor] Test set: Average loss: 0.3678, Accuracy: 16477/18800 (88%)
[hybrid_neg] Test set: Average loss: 0.3656, Accuracy: 16448/18800 (87%)
[synergy_nor] Test set: Average loss: 0.5036, Accuracy: 16594/18800 (88%)
[synergy_neg] Test set:

# Dodatni pomoćni elementi za FGSM

In [ ]:
loader_fgsm = emnist_loader()

test_loader_fgsm = loader_fgsm(batch_size=1)

# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    # get element-wise signs for gradient ascent
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    # clip to [0,1]
    # perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def test_fgsm(model, device, test_loader, epsilon):
    model.eval()
    correct = 0
    adv_examples = []

    for data, target in test_loader:

        data, target = data.to(device), target.to(device)
        data.requires_grad = True
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]

        # breakpoint()
        # If the initial prediction is wrong
        # dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        loss = F.nll_loss(output, target)

        model.zero_grad()

        loss.backward()

        data_grad = data.grad.data

        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        output = model(perturbed_data)

        final_pred = output.max(1, keepdim=True)[1]

        if final_pred.item() == target.item():
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))

    final_acc = correct / float(len(test_loader))
    print(
        "Model: {}\tEpsilon: {}\tTest Accuracy = {} / {} = {}".format(
            model.net_type, epsilon, correct, len(test_loader), final_acc
        )
    )

    return final_acc, adv_examples

# Testiranje za FGSM

In [ ]:
# Testing:

start_time = time.time()

models_fgsm = [
    model_normal,
    model_negative,
    model_hybrid_nor,
    model_hybrid_neg,
    model_synergy_nor,
    model_synergy_neg,
    model_synergy_all,
    model_tr_synergy_all
]

epsilons = [0, 0.01, 0.02, 0.03, 0.04, 0.05]
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    for model in models_fgsm:
        acc, ex = test_fgsm(model, device, test_loader_fgsm, eps)
        accuracies.append(acc)

print('--- Total time: %s seconds ---' % (time.time() - start_time))


Model: normal	Epsilon: 0	Test Accuracy = 16312 / 18800 = 0.8676595744680851
Model: negative	Epsilon: 0	Test Accuracy = 16419 / 18800 = 0.8733510638297872
Model: hybrid_nor	Epsilon: 0	Test Accuracy = 16449 / 18800 = 0.8749468085106383
Model: hybrid_neg	Epsilon: 0	Test Accuracy = 16459 / 18800 = 0.8754787234042554
Model: synergy_nor	Epsilon: 0	Test Accuracy = 16504 / 18800 = 0.8778723404255319
Model: synergy_neg	Epsilon: 0	Test Accuracy = 16591 / 18800 = 0.8825
Model: synergy_all	Epsilon: 0	Test Accuracy = 16625 / 18800 = 0.8843085106382979
Model: tr_synergy_all	Epsilon: 0	Test Accuracy = 16431 / 18800 = 0.8739893617021277
Model: normal	Epsilon: 0.01	Test Accuracy = 16147 / 18800 = 0.8588829787234042
Model: negative	Epsilon: 0.01	Test Accuracy = 16231 / 18800 = 0.8633510638297872
Model: hybrid_nor	Epsilon: 0.01	Test Accuracy = 16272 / 18800 = 0.865531914893617
Model: hybrid_neg	Epsilon: 0.01	Test Accuracy = 16263 / 18800 = 0.8650531914893617
Model: synergy_nor	Epsilon: 0.01	Test Accuracy

#Black-box PGD

In [ ]:
import torchattacks
from torchattacks import PGD
from torch.utils.data import DataLoader, TensorDataset

batch_size = 128

train_att = datasets.EMNIST(root='../data', split='balanced', train=True,
                      download=True, transform=transforms.ToTensor())
test_att = datasets.EMNIST(root='../data', split='balanced', train=False,
                     download=True, transform=transforms.ToTensor())

train_loader_att = torch.utils.data.DataLoader(train_att,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader_att = torch.utils.data.DataLoader(test_att,
                                          batch_size=batch_size,
                                          shuffle=False)

transform_att = transforms.Normalize((0.1307,), (0.3081,))

# loader_att = emnist_loader()

# test_loader_att = loader_att(batch_size=batch_size)

models_test = [
    model_normal,
    model_synergy_nor,
    model_synergy_all,
    model_tr_synergy_all
]

epsilons = [0.0, 0.01, 0.03, 0.05]


for epsilon in epsilons:
    print("=" * 30)
    print("epsilon =", epsilon)
    print("=" * 30)

    for attacked in models_test:
        print('attacked: ', attacked.net_type)
        pgd_attack = PGD(attacked, eps=epsilon, alpha=2/255, steps=40)
#        pgd_attack.set_return_type('int')
        pgd_attack.save(data_loader=test_loader_att, save_path="../data/emnist_pgd.pt",
                        verbose=False, save_type='int')

        adv_dict = torch.load("../data/emnist_pgd.pt")
        adv_images = adv_dict['adv_inputs']
        adv_labels = adv_dict['labels']

        adv_images = transform_att(adv_images.float()/255)
        adv_data = TensorDataset(adv_images, adv_labels)
        adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

        for model in models_test:
            model.eval()
            correct = 0
            total = 0

            for images, labels in adv_loader:
                images = images.cuda()
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels.cuda()).sum()

            print(model.net_type, 'acc: %.2f %%' % (100 * float(correct) / total), end='; ')
        print("\n")


epsilon = 0.0
attacked:  normal
normal acc: 86.77 %; synergy_nor acc: 87.79 %; synergy_all acc: 88.43 %; tr_synergy_all acc: 87.40 %; 

attacked:  synergy_nor
normal acc: 86.77 %; synergy_nor acc: 87.79 %; synergy_all acc: 88.43 %; tr_synergy_all acc: 87.40 %; 

attacked:  synergy_all
normal acc: 86.77 %; synergy_nor acc: 87.79 %; synergy_all acc: 88.43 %; tr_synergy_all acc: 87.40 %; 

attacked:  tr_synergy_all
normal acc: 86.77 %; synergy_nor acc: 87.79 %; synergy_all acc: 88.43 %; tr_synergy_all acc: 87.40 %; 

epsilon = 0.01
attacked:  normal
normal acc: 85.19 %; synergy_nor acc: 86.55 %; synergy_all acc: 87.29 %; tr_synergy_all acc: 86.12 %; 

attacked:  synergy_nor
normal acc: 84.99 %; synergy_nor acc: 86.12 %; synergy_all acc: 86.90 %; tr_synergy_all acc: 85.66 %; 

attacked:  synergy_all
normal acc: 85.11 %; synergy_nor acc: 86.14 %; synergy_all acc: 86.67 %; tr_synergy_all acc: 85.45 %; 

attacked:  tr_synergy_all
normal acc: 85.52 %; synergy_nor acc: 86.61 %; synergy_all acc: